In [1]:
# imports
import sys
import os
import socket
import pandas as pd
import glob

In [2]:
# function to format dates to datetime and replace nans in eventdate with sysdate
def formatdate(data):
    data['sysdate'] = data['sysdate'].str.replace('/', '')
    data['sysdate']= pd.to_datetime(data['sysdate'], format='%d%m%Y')
    data['eventdate']= data['eventdate'].str.replace('/', '')
    data['eventdate'] = pd.to_datetime(data['eventdate'], format='%d%m%Y')
    data['eventdate'].fillna(data['sysdate'], inplace = True)
    return data

In [3]:
next(os.walk('/mnt/lustre/users/maths/tws21/seb_tmp/bsms2730/bsms2730/Primary Care/CPRD GOLD'))

('/mnt/lustre/users/maths/tws21/seb_tmp/bsms2730/bsms2730/Primary Care/CPRD GOLD',
 [],
 ['19_050_gold_extract_therapy_072.txt',
  '19_050_gold_extract_test_010.txt',
  '19_050_gold_extract_therapy_055.txt',
  '19_050_gold_extract_therapy_113.txt',
  '19_050_gold_extract_therapy_033.txt',
  '19_050_gold_extract_therapy_077.txt',
  '19_050_gold_extract_therapy_023.txt',
  '19_050_gold_extract_clinical_028.txt',
  '19_050_gold_extract_test_018.txt',
  '19_050_gold_extract_test_027.txt',
  '19_050_gold_extract_clinical_010.txt',
  '19_050_gold_extract_test_012.txt',
  '19_050_gold_extract_additional_001.txt',
  '19_050_gold_extract_test_023.txt',
  '19_050_gold_extract_therapy_036.txt',
  '19_050_gold_extract_therapy_062.txt',
  '19_050_gold_extract_clinical_024.txt',
  '19_050_gold_extract_therapy_007.txt',
  '19_050_gold_extract_therapy_079.txt',
  '19_050_gold_extract_therapy_060.txt',
  '19_050_gold_extract_therapy_099.txt',
  '19_050_gold_extract_therapy_061.txt',
  '19_050_gold_extr

In [4]:
# read in dementia codes
dementia_codes = pd.read_excel('/mnt/maths/tws21/dementia_codes.xlsx')
dementia_codes.head()

,medcode,readcode,readterm,type,probable,possible,diagnosis,history,admin,assessment,suspected,referral,drugs
0,1917,F110.00,Alzheimer's disease,Alzheimer's,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,7664,Eu00.00,[X]Dementia in Alzheimer's disease,Alzheimer's,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,8195,Eu00z11,[X]Alzheimer's dementia unspec,Alzheimer's,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,11379,Eu00112,"[X]Senile dementia,Alzheimer's type",Alzheimer's,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,16797,F110000,Alzheimer's disease with early onset,Alzheimer's,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# select only diagnosis codes
dementia_med_codes = dementia_codes[dementia_codes['diagnosis'] == 1.0]['medcode']
dementia_med_codes

0       1917
1       7664
2       8195
3      11379
4      16797
       ...  
136    49513
137    53446
138    55222
139    55838
153    30706
Name: medcode, Length: 94, dtype: int64

In [6]:
# set the path of the file directory for CPRD Gold Primary Care data
path = '/mnt/lustre/users/maths/tws21/seb_tmp/bsms2730/bsms2730/Primary Care/CPRD GOLD/'

In [7]:
#read in and merge the referral files and format the date of each
referral_files = glob.glob(path + "*referral*.txt")

referral_li = []
for filename in referral_files:
    df = pd.read_csv(filename, sep="\t")
    df = df[df['medcode'].isin(list(dementia_med_codes))]
    df = formatdate(df)
    referral_li.append(df)

referral_merged = pd.concat(referral_li, axis=0)
referral_merged.head()

,e_patid,eventdate,sysdate,constype,consid,medcode,nhsspec,attendance,urgency
3074,10525191,2003-05-02,2003-05-02,4,1227502,6578,54,1,2
3079,10525191,2003-07-01,2003-07-01,4,1227532,6578,0,0,2
6221,11155213,2009-10-01,2009-10-02,4,805469,1916,70,0,2
6738,112629220,2006-05-03,2006-05-04,4,1845552,1350,54,1,4
9910,117980213,2014-03-07,2014-03-10,4,2482894,1350,41,0,2


In [8]:
# save as parquet
referral_merged.to_parquet(os.getcwd() + '/dementia_patients_referral_data.parquet')

In [9]:
referral_merged.shape[0]

5243